<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/CV_CNN/class_004_gridSearch_CNN_blood_cells_may_10_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [2]:
# !pip install optuna

In [3]:
from tensorboard.plugins.hparams import api as hp

In [4]:
#import optuna

In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import  Dropout,ZeroPadding2D, MaxPooling2D, Activation, add
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import cv2
import numpy as np

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

import zipfile
import pandas as pd


from sklearn.metrics import classification_report, confusion_matrix

In [6]:
IMAGE_SIZE = [224,224]
ImgSize = IMAGE_SIZE[0]
epochs = 16
batch_size = 128

In [7]:
!gdown 1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm

Downloading...
From: https://drive.google.com/uc?id=1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm
To: /content/blood_cell_images.zip
100% 110M/110M [00:02<00:00, 52.6MB/s] 


In [8]:
FILE='blood_cell_images.zip'

In [9]:
archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

In [10]:
PARAMS = {'lr':0.005, 
          'epochs':10,
          'batch_size':32}


In [11]:
!ls

blood_cell_images  blood_cell_images.zip  __MACOSX  sample_data


In [12]:
train_path = 'blood_cell_images/TRAIN'
test_path = 'blood_cell_images/TEST'
images_test = glob(test_path + '/*/*.jp*g')
images_train = glob(train_path + '/*/*.jp*g')

In [13]:
folders_train = glob(train_path + '/*')
folders_test = glob(test_path + '/*')

In [14]:
folders_train

['blood_cell_images/TRAIN/MONOCYTE',
 'blood_cell_images/TRAIN/NEUTROPHIL',
 'blood_cell_images/TRAIN/LYMPHOCYTE',
 'blood_cell_images/TRAIN/EOSINOPHIL']

In [15]:
folders_test

['blood_cell_images/TEST/MONOCYTE',
 'blood_cell_images/TEST/NEUTROPHIL',
 'blood_cell_images/TEST/LYMPHOCYTE',
 'blood_cell_images/TEST/EOSINOPHIL']

In [16]:
ntrain, ntest = len(images_train),len(images_test)
ntrain,ntest

(9957, 2487)

In [17]:
images_train[:4]
num_classes = 4

In [18]:
types = ['NEUTROPHIL', 'EOSINOPHIL', 'MONOCYTE', 'LYMPHOCYTE']

In [19]:
size = 20 # resize the images considering this value
n_epochs = 1000

In [20]:

def load_images(images,types=types, dim = (size,size)):

  nsize = len(images)
  X =[]
  y =[]
  names =[]
  k = 0

  for i in range(nsize): 
    img = image.load_img(images[i])
    im = cv2.resize(np.array(img), dim)
    im = preprocess_input(im)
    k += 1
    if( k % 3 == 0):
      X.append(im)

      if(types[0] in images[i]):
        names.append(types[0])
        y.append(0)
      elif(types[1] in images[i]):
        names.append(types[1])
        y.append(1)
      elif(types[2] in images[i]):
        names.append(types[2])
        y.append(2)
      else:
        names.append(types[3])
        y.append(3)

  X =np.array(X)
  return X,y,names

In [21]:

X_train, y_train, names_train = load_images(images_train)
X_test, y_test, names_test = load_images(images_test)

In [22]:
X_train[0].shape

(20, 20, 3)

In [23]:
df = pd.DataFrame({'name':names_train,'class':y_train})
y_train = pd.get_dummies(df['class'])
#y_train

In [24]:
for i in range(4): 
  df_plot= df[(df["class"] == i)]
  n= df_plot.shape[0]
  print(types[i],n)

NEUTROPHIL 833
EOSINOPHIL 833
MONOCYTE 826
LYMPHOCYTE 827


In [25]:
df = pd.DataFrame({'name':names_test,'class':y_test})
y_test = pd.get_dummies(df['class'])
#y_test

In [26]:
HP_DROPOUT = [0.0, 0.2]
HP_OPTIMIZER = ['adam', 'sgd','RMSprop']
HP_LEARNING_RATE = [ 1e-3] # [ 1e-3, 1e-4]
HP_EPOCHS = [10, 20]
METRIC_ACCURACY = 'accuracy'

In [35]:
def train_test_model(hparams):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(hparams['HP_DROPOUT']))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(hparams['HP_DROPOUT']))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(hparams['HP_DROPOUT']))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  if 'adam'== hparams['HP_OPTIMIZER']:
    optimizer = optimizer=tf.keras.optimizers.Adam(learning_rate=hparams['HP_LEARNING_RATE'])
  elif 'sgd'== hparams['HP_OPTIMIZER']:
    optimizer = optimizer=tf.keras.optimizers.SGD(learning_rate=hparams['HP_LEARNING_RATE'])
  else: # 'RMSprop'
    optimizer = optimizer=tf.keras.optimizers.RMSprop(learning_rate=hparams['HP_LEARNING_RATE']/10)

  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])  
  model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    epochs= hparams['HP_EPOCHS'],
    verbose=0) # verbose=2
  
  _, accuracy_test = model.evaluate(X_test,y_test)
  _, accuracy_train = model.evaluate(X_train,y_train)
  return accuracy_train, accuracy_test

In [36]:
session_num = 0
ans_model =[]
for learning_rate in HP_LEARNING_RATE:
  for optimizer in HP_OPTIMIZER:
    for dropout_rate in HP_DROPOUT:
      for epochs in HP_EPOCHS:

        hparams = {
            'HP_LEARNING_RATE': learning_rate,
            'HP_DROPOUT': dropout_rate,
            'HP_EPOCHS': epochs,
            'HP_OPTIMIZER': optimizer
            
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        acc_train, acc_test = train_test_model(hparams)
        ans_model.append([learning_rate, dropout_rate,epochs, optimizer,acc_train,acc_test])
        session_num += 1

--- Starting trial: run-0
104/104 [==============================] - 0s 3ms/step - loss: 0.6734 - accuracy: 0.7258
--- Starting trial: run-1
104/104 [==============================] - 0s 3ms/step - loss: 0.0350 - accuracy: 0.9973
--- Starting trial: run-2
104/104 [==============================] - 0s 3ms/step - loss: 0.5178 - accuracy: 0.8039
--- Starting trial: run-3
104/104 [==============================] - 0s 3ms/step - loss: 0.1032 - accuracy: 0.9886
--- Starting trial: run-4
104/104 [==============================] - 0s 3ms/step - loss: 1.3810 - accuracy: 0.2986
--- Starting trial: run-5
104/104 [==============================] - 0s 3ms/step - loss: 1.3771 - accuracy: 0.3013
--- Starting trial: run-6
104/104 [==============================] - 0s 3ms/step - loss: 1.3815 - accuracy: 0.2823
--- Starting trial: run-7
104/104 [==============================] - 0s 3ms/step - loss: 1.3799 - accuracy: 0.2886
--- Starting trial: run-8
104/104 [==============================] - 0s 4ms/step

In [37]:
df = pd.DataFrame(ans_model,columns=['learning_rate', 'dropout_rate','epochs', 'optimizer','acc_train','acc_test'])

In [38]:
df

,learning_rate,dropout_rate,epochs,optimizer,acc_train,acc_test
0,0.001,0.0,10,adam,0.725821,0.546441
1,0.001,0.0,20,adam,0.997288,0.676719
2,0.001,0.2,10,adam,0.803857,0.624849
3,0.001,0.2,20,adam,0.988551,0.710495
4,0.001,0.0,10,sgd,0.298584,0.265380
5,0.001,0.0,20,sgd,0.301296,0.275030
6,0.001,0.2,10,sgd,0.282314,0.244873
7,0.001,0.2,20,sgd,0.288641,0.277443
8,0.001,0.0,10,RMSprop,0.551672,0.443908
9,0.001,0.0,20,RMSprop,0.593552,0.457177


In [39]:
sorted_df = df.sort_values(by='acc_test', ascending=False)
print(sorted_df)

    learning_rate  dropout_rate  epochs optimizer  acc_train  acc_test
3           0.001           0.2      20      adam   0.988551  0.710495
1           0.001           0.0      20      adam   0.997288  0.676719
2           0.001           0.2      10      adam   0.803857  0.624849
0           0.001           0.0      10      adam   0.725821  0.546441
11          0.001           0.2      20   RMSprop   0.618259  0.522316
9           0.001           0.0      20   RMSprop   0.593552  0.457177
8           0.001           0.0      10   RMSprop   0.551672  0.443908
10          0.001           0.2      10   RMSprop   0.480868  0.435464
7           0.001           0.2      20       sgd   0.288641  0.277443
5           0.001           0.0      20       sgd   0.301296  0.275030
4           0.001           0.0      10       sgd   0.298584  0.265380
6           0.001           0.2      10       sgd   0.282314  0.244873
